In [1]:
import requests
import json
import pandas as pd
import openai
import os
import sys

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
response = requests.get('https://api.library.tulipintra.net/v2/assets')
body = response.content
body_json = json.loads(body)        

In [6]:
apps = body_json['apps']
app_groups = body_json['appGroups']
df_apps = pd.DataFrame(apps)
df_app_groups = pd.DataFrame(app_groups)

df_union = pd.concat([df_apps, df_app_groups])
print(df_union.columns)

#df_apps.columns
df_union_col = ['description','marketingHeadline', 'name', 'slug']
df_union = df_union[df_union_col] 
df_union['url'] = 'https://tulip.co/library/apps/'+df_union['slug']
df_union = df_union.drop(columns='slug')
df_union.tail()

Index(['assetId', 'assetType', 'created', 'description', 'href', 'imageUrl',
       'importance', 'lastModified', 'marketingHeadline', 'name', 'slug',
       'tags', 'thumbnailImageUrl', 'videoUrl'],
      dtype='object')


,description,marketingHeadline,name,url
13,The Total Productive Maintenance Suite assists...,Manage assets and procedures,Total Productive Maintenance App Suite,https://tulip.co/library/apps/total-productive...
14,"With the Digital Andon System, you can extend ...",Connect operators with supervisors in the case...,Digital Andon System,https://tulip.co/library/apps/digital-andon-sy...
15,Pack and Ship provides a lightweight solution ...,Track the packaging process and guide operator...,Pack and Ship,https://tulip.co/library/apps/pack-and-ship
16,The Tool and Asset Manager is a simple way to ...,The Tool and Asset Manager is a simple way to ...,Tool and Asset Manager,https://tulip.co/library/apps/tool-and-asset-m...
17,This app suite allows you to create new checkl...,"Create, manage, and track the completion of ta...",Checklist Management Suite,https://tulip.co/library/apps/checklist-manage...


In [14]:
df_apps.head()

,description,marketingHeadline,name,url
0,The Banner Unit Test app simplifies the integr...,Integrate Banner Andon lights to your Tulip apps,Banner Unit Test,https://tulip.co/library/apps/banner-unit-test
1,The Stopwatch Widget Unit Test adds a basic ti...,Track time within an app,Stopwatch Widget Unit Test,https://tulip.co/library/apps/stopwatch-widget...
2,This functional Example application shows you ...,Use UMH and Tulip together bridge the human-ma...,United Manufacturing Hub Functional Example,https://tulip.co/library/apps/united-manufactu...
3,The Radial Gauge Widget Unit Test allows a use...,Visualize if measurements and values meet acce...,Radial Gauge Widget,https://tulip.co/library/apps/radial-gauge-widget
4,The Inventory Management app is intended to sh...,"Store inventory levels, adjust quantities and ...",Inventory Management,https://tulip.co/library/apps/inventory-manage...


In [15]:
df_apps_json = df_apps.to_json(orient='split')

In [16]:
question = "I am looking for an app that I can use for improving machine uptime; what would you recommend?"

In [28]:
prompt = f"""please provide an answer to the question belowing use the DataFrame {df_apps_json} as context. 
You are an AI library assistant skilled in manufacturing and frontline operations. Based upon the question, please recommend some of the apps provided in the json dataframe. 
Please provide some context on the recommendation, and the url for viewing. Please provide around 5 recommended app sorted by relevance. Question below:
{question}
"""

In [29]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo-16k',
    messages = [{"role":"user", "content": prompt}],
    max_tokens = 2000
)

In [34]:
print(response.choices[0].message.content)

Based on your requirements for improving machine uptime, I would recommend the following apps from the provided DataFrame:

1. App Name: Machine Monitoring Dashboard
   Description: Gain real-time insights into production to identify ways to improve operations. Monitor machines, track status, and visualize performance across your facilities.
   URL: [Machine Monitoring Dashboard](https://tulip.co/library/apps/machine-monitoring-dashboard)

2. App Name: Performance Visibility Dashboard
   Description: Gain control over your operations by tracking key operational metrics in real-time. Visually manage how you are performing in key areas across your operation.
   URL: [Performance Visibility Dashboard](https://tulip.co/library/apps/performance-visibility-dashboard)

3. App Name: Performance Visibility Terminal
   Description: Capture performance data and log part production. Provide a tool for logging data and further analysis.
   URL: [Performance Visibility Terminal](https://tulip.co/lib